In [ ]:
import urllib.request
import zipfile
import tensorflow as tf
import os
from keras_preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import RMSprop


def solution_A2():
    data_url_1 = 'https://github.com/dicodingacademy/assets/releases/download/release-horse-or-human/horse-or-human.zip'
    urllib.request.urlretrieve(data_url_1, 'horse-or-human.zip')
    local_file = 'horse-or-human.zip'
    zip_ref = zipfile.ZipFile(local_file, 'r')
    zip_ref.extractall('data/horse-or-human')

    data_url_2 = 'https://github.com/dicodingacademy/assets/raw/main/Simulation/machine_learning/validation-horse-or-human.zip'
    urllib.request.urlretrieve(data_url_2, 'validation-horse-or-human.zip')
    local_file = 'validation-horse-or-human.zip'
    zip_ref = zipfile.ZipFile(local_file, 'r')
    zip_ref.extractall('data/validation-horse-or-human')
    zip_ref.close()

    TRAINING_DIR = 'data/horse-or-human'
    VALIDATION_DIR = 'data/validation-horse-or-human'

    train_datagen = ImageDataGenerator(
        rescale=1./255,
        rotation_range=20,
        shear_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest'
    )

    train_generator = train_datagen.flow_from_directory(
        TRAINING_DIR,
        target_size=(150, 150),
        batch_size=32,
        class_mode='binary'
    )

    validation_generator = train_datagen.flow_from_directory(
        VALIDATION_DIR,
        target_size=(150, 150),
        batch_size=32,
        class_mode='binary'
    )

    model = tf.keras.models.Sequential([
        tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
        tf.keras.layers.MaxPooling2D(2, 2),
        tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
        tf.keras.layers.MaxPooling2D(2, 2),
        tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
        tf.keras.layers.MaxPooling2D(2, 2),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(512, activation='relu'),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(1, activation='sigmoid')
    ])

    model.compile(loss='binary_crossentropy',
                  optimizer=RMSprop(learning_rate=0.001),
                  metrics=['accuracy'])

    class myCallback(tf.keras.callbacks.Callback):
        def on_epoch_end(self, epoch, logs={}):
            if logs.get('accuracy') > 0.83 and logs.get('val_accuracy') > 0.83:
                print("\nTarget akurasi tercapai! Pelatihan dihentikan.")
                self.model.stop_training = True

    callbacks = myCallback()

    history = model.fit(
        train_generator,
        steps_per_epoch=len(train_generator),
        epochs=30,
        validation_data=validation_generator,
        validation_steps=len(validation_generator),
        callbacks=[callbacks],
        verbose=1
    )

    return model


if __name__ == '__main__':
    model = solution_A2()
    model.save("model_A2.h5")


Found 1027 images belonging to 2 classes.
Found 256 images belonging to 2 classes.
Epoch 1/30
33/33 [==============================] - 47s 1s/step - loss: 0.8811 - accuracy: 0.6796 - val_loss: 0.5373 - val_accuracy: 0.8477
Epoch 2/30
33/33 [==============================] - 46s 1s/step - loss: 0.3448 - accuracy: 0.8666 - val_loss: 0.5381 - val_accuracy: 0.8203
Epoch 3/30
33/33 [==============================] - 46s 1s/step - loss: 0.2891 - accuracy: 0.9056 - val_loss: 1.0122 - val_accuracy: 0.7578
Epoch 4/30
33/33 [==============================] - 47s 1s/step - loss: 0.2845 - accuracy: 0.9279 - val_loss: 1.5888 - val_accuracy: 0.7539
Epoch 5/30
33/33 [==============================] - ETA: 0s - loss: 0.1074 - accuracy: 0.9542
Target akurasi tercapai! Pelatihan dihentikan.
33/33 [==============================] - 47s 1s/step - loss: 0.1074 - accuracy: 0.9542 - val_loss: 0.6970 - val_accuracy: 0.8750
